## 1. Préparer les données

Les données ne sont pas toujours prêtes à être utilisées avec un modèle d'apprentissage automatique Scikit-Learn.

Trois des principales étapes que vous devrez souvent suivre sont :
* Diviser les données en fonctionnalités (généralement « X ») et étiquettes (généralement « y »).
* Diviser les données en ensembles de formation et de test (et éventuellement un ensemble de validation).
* Remplissage (également appelé imputation) ou non-respect des valeurs manquantes.
* Conversion de valeurs non numériques en valeurs numériques (appelé également codage de fonctionnalités).

Voyons un exemple.

In [1]:
import datetime
import numpy as np
import pandas as pd
heart_disease = pd.read_csv('../data/heart-disease.csv')

In [2]:
# Diviser les données en X et y
heart_disease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
# Diviser les données en fonctionnalités (X) et étiquettes (y)
X = heart_disease.drop('target', axis=1)
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


Bon! On dirait que notre ensemble de données contient 303 échantillons avec 13 fonctionnalités (13 colonnes).

Vérifions les étiquettes.

In [4]:
y = heart_disease['target']
y

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64

Magnifique, 303 étiquettes avec des valeurs de « 0 » (pas de maladie cardiaque) et « 1 » (maladie cardiaque).

Divisons maintenant nos données en ensembles d'entraînement et de test, nous utiliserons une répartition 80/20 (80 % des échantillons pour l'entraînement et 20 % des échantillons pour les tests).

In [5]:
# Diviser les données en ensembles de formation et de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2) # vous pouvez modifier la taille du test

# Vérifiez les formes des différentes répartitions de données
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((242, 13), (61, 13), (242,), (61,))

In [6]:
# 80 % des données sont utilisées pour l'ensemble de formation (le modèle apprendra des modèles sur ces échantillons)
X.shape[0] * 0.8

242.4

In [7]:
# Et 20 % des données sont utilisées pour l'ensemble de tests (le modèle sera évalué sur ces échantillons)
X.shape[0] * 0.2

60.6

### 1.1 Assurez-vous que tout est numérique

Les ordinateurs adorent les chiffres.

Donc, une chose dont vous devrez souvent vous assurer est que vos ensembles de données sont sous forme numérique.

Cela vaut même pour les ensembles de données contenant des caractéristiques non numériques que vous souhaiterez peut-être inclure dans un modèle.

Par exemple, si nous travaillions avec un ensemble de données sur les ventes de voitures, comment pourrions-nous transformer des fonctionnalités telles que « Marque » et « Couleur » en nombres ?

Voyons cela.

Tout d'abord, nous allons importer l'ensemble de données « car-sales-extended.csv ».

In [8]:
# Importer car-sales-extended.csv
car_sales = pd.read_csv("../data/car-sales-extended.csv")
car_sales

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431,4,15323
1,BMW,Blue,192714,5,19943
2,Honda,White,84714,4,28343
3,Toyota,White,154365,4,13434
4,Nissan,Blue,181577,3,14043
...,...,...,...,...,...
995,Toyota,Black,35820,4,32042
996,Nissan,White,155144,3,5716
997,Nissan,Blue,66604,4,31570
998,Honda,White,215883,4,4001


Nous pouvons vérifier les types d'ensembles de données avec `.dtypes`.

In [9]:
car_sales.dtypes

Make             object
Colour           object
Odometer (KM)     int64
Doors             int64
Price             int64
dtype: object

Notez que les fonctionnalités `Make` et `Color` sont de `dtype=object` (ce sont des chaînes) alors que le reste des colonnes sont de `dtype=int64`.

Si nous voulons utiliser les fonctionnalités « Make » et « Color » dans notre modèle, nous devrons trouver comment les transformer sous forme numérique.

In [10]:
# Divisé en X & y et entraînement/test
X = car_sales.drop("Price", axis=1)
y = car_sales["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Essayons maintenant de construire un modèle sur nos données `car_sales`.

In [11]:
# Essayez de prédire avec une forêt aléatoire sur la colonne des prix (ne fonctionne pas)
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)
model.score(X_test, y_test)

ValueError: could not convert string to float: 'Honda'

Oups... cela ne fonctionne pas, nous devrons d'abord convertir les caractéristiques non numériques en nombres.

Le processus de transformation de caractéristiques catégorielles en nombres est souvent appelé **encodage**.

Scikit-Learn propose un fantastique guide détaillé sur [*Encodage des fonctionnalités catégorielles*](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features).

Mais examinons l'un des moyens les plus simples de transformer des caractéristiques catégorielles en nombres, [encodage à chaud](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).

En apprentissage automatique, [one-hot encoding](https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics) donne une valeur de « 1 » à la valeur cible et une valeur de « 0 » à l'autre valeurs.

Par exemple, disons que nous avions cinq échantillons et trois options de marques de voitures, Honda, Toyota, BMW.

Et nos échantillons étaient :
1.Honda
2.BMW
3.BMW
4.Toyota
5.Toyota

Si nous devions les encoder à chaud, cela ressemblerait à :

| Sample | Honda | Toyota | BMW |
| ----- | ----- | ----- | ----- |
| 1 | 1 | 0 | 0 |
| 2 | 0 | 0 | 1 |
| 3 | 0 | 0 | 1 |
| 4 | 0 | 1 | 0 |
| 5 | 0 | 1 | 0 |

Remarquez qu'il y a un 1 pour chaque valeur cible mais un 0 pour chaque autre valeur.

Nous pouvons utiliser les étapes suivantes pour encoder à chaud notre ensemble de données :
1. Importez [`sklearn.preprocessing.OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) pour encoder à chaud nos fonctionnalités et [`sklearn.compose .ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) pour cibler les colonnes spécifiques de notre DataFrame à transformer.
2. Définissez les fonctionnalités catégorielles que nous souhaitons transformer.
3. Créez une instance de `OneHotEncoder`.
4. Créez une instance de `ColumnTransformer` et introduisez-lui les transformations que nous aimerions effectuer.
5. Ajustez l'instance de `ColumnTransformer` à nos données et transformez-la avec le [`fit_transform(X)`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html #sklearn.compose.ColumnTransformer.fit_transform).

> **Remarque :** Dans Scikit-Learn, le terme « transformateur » est souvent utilisé pour désigner quelque chose qui *transforme* les données.


In [13]:
# 1. Importez OneHotEncoder et ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# 2. Définir les fonctionnalités catégorielles à transformer
categorical_features = ["Make", "Colour", "Doors"]

# 3. Créez une instance de OneHotEncoder
one_hot = OneHotEncoder()

# 4. Créez une instance de ColumnTransformer
transformer = ColumnTransformer([("one_hot", # nom
                                  one_hot, # transformer
                                  categorical_features)], # colonnes à transformer
                                  remainder="passthrough") # que faire du reste des colonnes ? ("passthrough" = laisser inchangé)
# 5. Transformez les caractéristiques catégorielles en nombres (cela renverra une matrice clairsemée de type tableau, pas un DataFrame)
transformed_X = transformer.fit_transform(X)
transformed_X

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 3.54310e+04],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        1.00000e+00, 1.92714e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 8.47140e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 6.66040e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.15883e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.48360e+05]])

> **Remarque :** Vous vous demandez peut-être pourquoi nous avons considéré « Doors » comme une variable catégorielle. Ce qui est une bonne question étant donné que « Doors » est déjà numérique. Eh bien, la réponse est que les « portes » peuvent être numériques ou catégoriques. Cependant, j'ai décidé d'être catégorique, car d'où je viens, le nombre de portes est souvent une *catégorie* de voiture différente. Par exemple, vous pouvez acheter des voitures à 4 portes ou acheter des voitures à 5 portes (ce qui m'a toujours dérouté puisque où est la 5ème porte ?). Cependant, vous pouvez essayer de traiter cette valeur comme numérique ou catégorique, entraîner un modèle sur chacune, puis voir les performances de chaque modèle.

Waouh ! On dirait que nos échantillons sont tous numériques, à quoi ressemblaient nos données auparavant ?

In [14]:
X.head()

,Make,Colour,Odometer (KM),Doors
0,Honda,White,35431,4
1,BMW,Blue,192714,5
2,Honda,White,84714,4
3,Toyota,White,154365,4
4,Nissan,Blue,181577,3


Il semble que « OneHotEncoder » et « ColumnTransformer » aient transformé tous nos échantillons de données en nombres.

Voyons le premier échantillon transformé.

In [15]:
# Afficher le premier échantillon transformé
transformed_X[0]

array([0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       1.0000e+00, 0.0000e+00, 3.5431e+04])

Et quelles étaient ces valeurs à l’origine ?

In [16]:
# Afficher le premier échantillon original
X.iloc[0]

Make             Honda
Colour           White
Odometer (KM)    35431
Doors                4
Name: 0, dtype: object

#### 1.1.1 Codage numérique des données avec des pandas

Une autre façon de coder numériquement les données est directement avec les pandas.

Nous pouvons utiliser la méthode [`pandas.get_dummies()`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) (ou `pd.get_dummies()` pour faire court) et puis transmettez-lui nos colonnes cibles.

En retour, nous obtiendrons une version codée à chaud de nos colonnes cibles.

Rappelons-nous à quoi ressemble notre DataFrame.

In [17]:
# Afficher la tête du DataFrame d'origine
car_sales.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431,4,15323
1,BMW,Blue,192714,5,19943
2,Honda,White,84714,4,28343
3,Toyota,White,154365,4,13434
4,Nissan,Blue,181577,3,14043


Merveilleux, utilisons maintenant `pd.get_dummies()` pour transformer nos variables catégorielles en variables codées à chaud.

In [18]:
# Variables catégorielles d'encodage One-hot
categorical_variables = ["Make", "Colour", "Doors"]
dummies = pd.get_dummies(data=car_sales[categorical_variables])
dummies

,Doors,Make_BMW,Make_Honda,Make_Nissan,Make_Toyota,Colour_Black,Colour_Blue,Colour_Green,Colour_Red,Colour_White
0,4,0,1,0,0,0,0,0,0,1
1,5,1,0,0,0,0,1,0,0,0
2,4,0,1,0,0,0,0,0,0,1
3,4,0,0,0,1,0,0,0,0,1
4,3,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
995,4,0,0,0,1,1,0,0,0,0
996,3,0,0,1,0,0,0,0,0,1
997,4,0,0,1,0,0,1,0,0,0
998,4,0,1,0,0,0,0,0,0,1


Bon!

Remarquez qu'il y a une nouvelle colonne pour chaque option catégorielle (par exemple `Make_BMW`, `Make_Honda`, etc.).

Mais remarquez également comment il a également manqué la colonne « Portes » ?

C'est parce que `Doors` est déjà numérique, donc pour que `pd.get_dummies()` fonctionne dessus, nous pouvons le changer pour taper `object`.

Par défaut, `pd.get_dummies()` transforme également toutes les valeurs en bools (`True` ou `False`).

Nous pouvons obtenir les valeurs renvoyées sous la forme « 0 » ou « 1 » en définissant « dtype=float ».

In [19]:
# Je dois convertir les portes en objets pour que les nuls puissent travailler dessus...
car_sales["Doors"] = car_sales["Doors"].astype(object)
dummies = pd.get_dummies(data=car_sales[["Make", "Colour", "Doors"]],
                         dtype=float)
dummies

,Make_BMW,Make_Honda,Make_Nissan,Make_Toyota,Colour_Black,Colour_Blue,Colour_Green,Colour_Red,Colour_White,Doors_3,Doors_4,Doors_5
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
997,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


Nous avons maintenant transformé nos données sous une forme entièrement numérique à l'aide de Scikit-Learn et de pandas.

Maintenant, vous vous demandez peut-être...

**Devriez-vous utiliser Scikit-Learn ou pandas pour transformer les données sous forme numérique ?**

Et la réponse est soit.

Mais en règle générale :
* Si vous effectuez une **analyse rapide des données et exécutez de petites expériences de modélisation**, utilisez `pandas` car il est généralement assez rapide à démarrer.
* Si vous effectuez une **expérience de modélisation à plus grande échelle** ou si vous souhaitez intégrer vos **étapes de traitement des données dans un pipeline de production**, je vous recommande de vous tourner vers Scikit-Learn, en particulier un [Pipeline Scikit-Learn ](https://scikit-learn.org/stable/modules/compose.html#pipeline) (enchaînant plusieurs étapes d'estimateur/modélisation).

Puisque nous avons transformé nos données sous forme numérique, que diriez-vous d'essayer à nouveau d'ajuster notre modèle ?

Recréons une répartition train/test, sauf que cette fois nous utiliserons `transformed_X` au lieu de `X`.

In [20]:
np.random.seed(42)

# Create train and test splits with transformed_X
X_train, X_test, y_train, y_test = train_test_split(transformed_X,
                                                    y,
                                                    test_size=0.2)

# Create the model instance
model = RandomForestRegressor()

# Fit the model on the numerical data (this errored before since our data wasn't fully numeric)
model.fit(X_train, y_train)

# Score the model (returns r^2 metric by default, also called coefficient of determination, higher is better)
model.score(X_test, y_test)

0.3235867221569877

### 1.2 Que faire s'il manquait des valeurs dans les données ?

Des trous dans les données signifient des trous dans les modèles que votre modèle d'apprentissage automatique peut apprendre.

De nombreux modèles d'apprentissage automatique ne fonctionnent pas correctement ou génèrent des erreurs lorsqu'ils sont utilisés sur des ensembles de données comportant des valeurs manquantes.

Une valeur manquante peut apparaître sous forme d'espace, sous forme de « NaN » ou quelque chose de similaire.

Il existe deux options principales lorsqu'il s'agit de valeurs manquantes :

1. **Remplissez-les avec une valeur donnée ou calculée (imputation)** - Par exemple, vous pouvez remplir les valeurs manquantes d'une colonne numérique avec la moyenne de toutes les autres valeurs. La pratique consistant à calculer ou à déterminer comment remplir les valeurs manquantes dans un ensemble de données est appelée **imputation**. Pour une excellente ressource sur l'imputation des valeurs manquantes, je vous recommande de vous référer au [guide de l'utilisateur Scikit-Learn](https://scikit-learn.org/stable/modules/impute.html).
2. **Supprimez-les** - Si une ligne ou un échantillon comporte des valeurs manquantes, vous pouvez choisir de les supprimer complètement de votre ensemble de données. Cependant, cela entraîne potentiellement l'utilisation de moins de données pour créer votre modèle.

> **Remarque :** La gestion des valeurs manquantes diffère d'un problème à l'autre, ce qui signifie qu'il n'existe pas de meilleure façon à 100 % de combler les valeurs manquantes dans les ensembles de données et les types de problèmes. Il faudra souvent une expérimentation et une pratique minutieuses pour trouver la meilleure façon de gérer les valeurs manquantes dans vos propres ensembles de données.

Pour nous entraîner à gérer les valeurs manquantes, importons une version de l'ensemble de données « car_sales » avec plusieurs valeurs manquantes.

In [21]:
# Importer une base de données de ventes de voitures avec des valeurs manquantes
car_sales_missing = pd.read_csv("../data/car-sales-extended-missing-data.csv")
car_sales_missing

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


Si votre ensemble de données est volumineux, il est probable que vous n'allez pas le parcourir échantillon par échantillon pour trouver les valeurs manquantes.

Heureusement, pandas a une méthode appelée [`pd.DataFrame.isna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html) qui est capable de détecter les valeurs manquantes. .

Essayons-le sur notre DataFrame.

In [22]:
# Obtenez la somme de toutes les valeurs manquantes
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

Hmm... il semble qu'il y ait environ 50 valeurs manquantes par colonne.

Que diriez-vous d'essayer de diviser les données en fonctionnalités et étiquettes, puis de convertir les données catégorielles en nombres, puis de diviser les données en formation et test, puis d'essayer d'y adapter un modèle (comme nous l'avons fait auparavant) ?

In [23]:
# Créer des fonctionnalités
X_missing = car_sales_missing.drop("Price", axis=1)
print(f"Number of missing X values:\n{X_missing.isna().sum()}")

Number of missing X values:
Make             49
Colour           50
Odometer (KM)    50
Doors            50
dtype: int64


In [24]:
# Créer des labels
y_missing = car_sales_missing["Price"]
print(f"Number of missing y values: {y_missing.isna().sum()}")

Number of missing y values: 50


Nous pouvons maintenant convertir les colonnes catégorielles en encodages uniques, One-hot, (comme avant).

In [25]:
# Convertissons les colonnes catégorielles en une colonne codée à chaud (code copié ci-dessus)
# Transformez les catégories (Make et Colour) en nombres
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ["Make", "Colour", "Doors"]

one_hot = OneHotEncoder()

transformer = ColumnTransformer([("one_hot", 
                                  one_hot, 
                                  categorical_features)],
                                remainder="passthrough",
                                sparse_threshold=0) # return a sparse matrix or not

transformed_X_missing = transformer.fit_transform(X_missing)
transformed_X_missing

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 3.54310e+04],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.92714e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 8.47140e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 6.66040e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 2.15883e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 2.48360e+05]])

Enfin, divisons les échantillons de données manquantes en ensembles d'entraînement et de test, puis essayons d'ajuster et d'évaluer un modèle sur eux. 

In [26]:
# Diviser les données en ensembles de formation et de test
X_train, X_test, y_train, y_test = train_test_split(transformed_X_missing,
                                                    y_missing,
                                                    test_size=0.2)

# Ajuster et marquer un modèle
model = RandomForestRegressor()
model.fit(X_train, y_train)
model.score(X_test, y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

Mince ! Il semble que le modèle que nous essayons d'utiliser ne fonctionne pas avec des valeurs manquantes.

Lorsque nous essayons de l'adapter à un ensemble de données avec des échantillons manquants, Scikit-Learn produit l'erreur :

`ValueError : L'entrée X contient NaN. RandomForestRegressor n'accepte pas les valeurs manquantes codées nativement en NaN...`

On dirait que si nous voulons utiliser `RandomForestRegressor`, nous devrons soit remplir, soit supprimer les valeurs manquantes.

<details class="tip">
    <summary><strong>Note:</strong> Scikit-Learn dispose d'une 
        <a href="https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values">liste des modèles capables de gérer directement les NaN ou les valeurs manquantes</a>.
    </summary>
    <p>Comme, 
        <a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html"><code>sklearn.ensemble.HistGradientBoostingClassifier</code></a> 
        où 
        <a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html"><code>sklearn.ensemble.HistGradientBoostingRegressor</code></a>.
    </p>
    <p>À titre expérimental, vous voudrez peut-être essayer ce qui suit:</p>
    <pre><code>
from sklearn.ensemble import HistGradientBoostingRegressor

\# Essayez un modèle capable de gérer les NaN de manière native
nan_model = HistGradientBoostingRegressor()
nan_model.fit (X_train, y_train)
nan_model.score(X_test, y_test)
    </code></pre>
</details>
Voyons à nouveau quelles valeurs manquent.

In [ ]:
car_sales_missing.isna().sum()

Comment peut-on les combler (imputer) ou les supprimer ?

### 1.2.1 Remplissez les données manquantes avec des pandas

Voyons comment nous pourrions combler les valeurs manquantes avec des pandas.

Pour les valeurs catégorielles, l'un des moyens les plus simples consiste à remplir les champs manquants avec la chaîne « manquant » .

Nous pourrions faire cela pour les fonctionnalités « Make » et « Colour ».

Quant à la fonctionnalité « Portes », nous pourrions utiliser « « manquant » » ou nous pourrions la remplir avec l'option la plus courante « 4 ».

Avec la fonction « Odomètre (KM) », nous pouvons utiliser la valeur moyenne de toutes les autres valeurs de la colonne.

Et enfin, pour les échantillons pour lesquels il manque une valeur « Prix », nous pouvons les supprimer (puisque « Prix » est la valeur cible, la suppression cause probablement moins de dommages que l'imputation, cependant, vous pouvez concevoir une expérience pour tester cela).

En résumé:

| Colonne/Feature | Remplissez la valeur manquante avec | 
| ----- | ----- |
| `Make` | `"missing"` |
| `Colour` | `"missing"` |
| `Doors` | 4 (valeur la plus courante) |
| `Odometer (KM)` | moyenne `Odometer (KM)` | 
| `Price` (cible) | NA, supprimer les échantillons manquants `Price` |

> **Remarque :** La pratique consistant à remplir les données manquantes avec des valeurs données ou calculées est appelée [**imputation**](https://scikit-learn.org/stable/modules/impute.html). Et il est important de se rappeler qu’il n’existe pas de moyen idéal pour combler les données manquantes (à moins que ce ne soit avec des données qui auraient dû être là en premier lieu). Les méthodes que nous utilisons ne sont qu’une parmi tant d’autres. Les techniques que vous utiliserez dépendront fortement de votre ensemble de données. Un bon endroit où chercher serait la recherche de « techniques d'imputation de données ».

Commençons par la colonne `Make`.

Nous pouvons utiliser la méthode pandas [`fillna(value="missing", inplace=True)`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) pour tout remplir les valeurs manquantes avec la chaîne `"missing"`.

In [28]:
# Remplissez les valeurs manquantes dans la colonne Make
car_sales_missing["Make"].fillna(value="missing", inplace=True)

Et nous pouvons faire la même chose avec la colonne « Couleur ».

In [29]:
# Remplissez la colonne Couleur
car_sales_missing["Colour"].fillna(value="missing", inplace=True)

Combien de valeurs manquantes avons-nous maintenant ?

In [30]:
car_sales_missing.isna().sum()

Make              0
Colour            0
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

Merveilleux! Nous faisons quelques progrès.

Remplissons maintenant la colonne « Portes » avec « 4 » (la valeur la plus courante), cela revient à la remplir avec la [médiane](https://pandas.pydata.org/docs/reference/api/pandas. DataFrame.median.html) ou [mode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html) de la colonne `Doors`.

In [31]:
# Trouvez la valeur la plus courante de la colonne Doors
car_sales_missing["Doors"].value_counts()

4.0    811
5.0     75
3.0     64
Name: Doors, dtype: int64

In [32]:
# Remplissez la colonne Doors avec la valeur la plus courante
car_sales_missing["Doors"].fillna(value=4, inplace=True)

Ensuite, nous remplirons la colonne « Odometer (KM) » avec sa valeur moyenne.

In [33]:
# Remplissez la colonne Odometer (KM)
car_sales_missing["Odometer (KM)"].fillna(value=car_sales_missing["Odometer (KM)"].mean(), inplace=True)

Combien de valeurs manquantes avons-nous maintenant ?

In [34]:
# Vérifiez le nombre de valeurs manquantes
car_sales_missing.isna().sum()

Make              0
Colour            0
Odometer (KM)     0
Doors             0
Price            50
dtype: int64

Super ! Ça a l'air beaucoup mieux.

Enfin, nous pouvons supprimer les lignes pour lesquelles il manque la valeur cible « Prix ».

> **Remarque :** Une autre option consisterait à imputer la valeur du « Price » avec la moyenne ou la médiane ou une autre valeur calculée (par exemple en utilisant des voitures similaires pour estimer le prix), cependant, pour garder les choses simples et éviter d'introduire trop de fausses étiquettes dans les données, nous supprimerons les échantillons manquant d'une valeur « Price ».

In [35]:
# Supprimer les lignes avec des labels de prix manquantes
car_sales_missing.dropna(inplace=True)

That should be no more missing values!

In [36]:
# Vérifiez le nombre de valeurs manquantes
car_sales_missing.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

Depuis que nous avons supprimé les échantillons manquant d'une valeur « Prix », il y a maintenant moins d'échantillons globaux, mais aucun d'entre eux n'a de valeur manquante.

In [37]:
# Vérifiez le nombre total d'échantillons (auparavant, il était de 1 000)
len(car_sales_missing)

950

Pouvons-nous adapter un modèle maintenant ?

Essayons!

Nous allons d’abord créer les fonctionnalités et les étiquettes.

Ensuite, nous convertirons les variables catégorielles en nombres via un encodage à chaud.

Ensuite, nous diviserons les données en ensembles d'entraînement et de test, comme auparavant.

Enfin, nous essaierons d'adapter un modèle `RandomForestRegressor()` aux données nouvellement remplies.

In [38]:
# Créer des fonctionnalités ou features
X_missing = car_sales_missing.drop("Price", axis=1)
print(f"Number of missing X values:\n{X_missing.isna().sum()}")

# Créer des labels
y_missing = car_sales_missing["Price"]
print(f"Number of missing y values: {y_missing.isna().sum()}")

Number of missing X values:
Make             0
Colour           0
Odometer (KM)    0
Doors            0
dtype: int64
Number of missing y values: 0


In [39]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ["Make", "Colour", "Doors"]

one_hot = OneHotEncoder()

transformer = ColumnTransformer([("one_hot", 
                                  one_hot, 
                                  categorical_features)],
                                remainder="passthrough",
                                sparse_threshold=0) # renvoie ou non une matrice clairsemée

transformed_X_missing = transformer.fit_transform(X_missing)
transformed_X_missing

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 3.54310e+04],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        1.00000e+00, 1.92714e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 8.47140e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 6.66040e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.15883e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.48360e+05]])

In [40]:
# Diviser les données en ensembles de formation et de test
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(transformed_X_missing,
                                                    y_missing,
                                                    test_size=0.2)

# Ajuster et évaluer un modèle
model = RandomForestRegressor()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.22011714008302485

On dirait que remplir les valeurs manquantes avec des pandas a fonctionné !

Notre modèle peut être adapté aux données sans problème.

### 1.2.2 Remplissage des données manquantes et transformation des données catégorielles avec Scikit-Learn

Maintenant que nous avons rempli les colonnes manquantes à l'aide des fonctions pandas, vous vous demandez peut-être : « Pourquoi les pandas ? Je pensais que c'était une introduction à Scikit-Learn ?

Ne vous inquiétez pas, Scikit-Learn fournit une classe appelée [`sklearn.impute.SimpleImputer()`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) qui nous permet faire une chose similaire.

`SimpleImputer()` transforme les données en remplissant les valeurs manquantes avec un paramètre `stratégie` donné.

Et nous pouvons l'utiliser pour remplir les valeurs manquantes dans notre DataFrame comme ci-dessus.

Pour le moment, notre dataframe n'a aucune valeur manquante.

In [41]:
car_sales_missing.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

Réimportons-le pour qu'il contienne des valeurs manquantes et que nous puissions les remplir avec Scikit-Learn.

In [42]:
# Réimporter le DataFrame
car_sales_missing = pd.read_csv("../data/car-sales-extended-missing-data.csv")
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

Pour commencer, nous allons supprimer les lignes pour lesquelles il manque une valeur « Price ».

In [43]:
# Supprimez les lignes manquantes dans la colonne Price
car_sales_missing.dropna(subset=["Price"], inplace=True)

Désormais, il ne manque plus aucune ligne contenant une valeur « Price ».

In [44]:
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

Puisque nous n'avons pas besoin de renseigner de valeurs « Price », divisons nos données en fonctionnalités (`X`) et étiquettes (`y`).

Nous diviserons également les données en ensembles de formation et de test.

In [45]:
# Divisé en X et y
X = car_sales_missing.drop("Price", axis=1)
y = car_sales_missing["Price"]

# Diviser les données en train et test
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2)

> **Remarque :** Nous avons d'abord divisé les données en ensembles d'entraînement et de test pour remplir séparément les valeurs manquantes. Il s'agit d'une bonne pratique car l'ensemble de test est censé émuler des données que le modèle n'a jamais vues auparavant. Pour les variables catégorielles, il est généralement acceptable de remplir des valeurs sur l'ensemble de l'ensemble de données. Cependant, pour les variables numériques, vous devez **remplir uniquement les valeurs de l'ensemble de test qui ont été calculées à partir de l'ensemble d'entraînement**.

Ensembles de formation et de test créés !

Configurons maintenant quelques instances de `SimpleImputer()` pour remplir diverses valeurs manquantes.

Nous utiliserons les stratégies suivantes et remplirons les valeurs :
* Pour les colonnes catégorielles (`Make`, `Color`), `strategy="constant"`, `fill_value="missing"` (remplissez les échantillons manquants avec une valeur cohérente de `"missing"`.
* Pour la colonne `Door`, `strategy="constant"`, `fill_value=4` (remplissez les échantillons manquants avec une valeur cohérente de `4`).
* Pour la colonne numérique (`Odometer (KM)`), `strategy="mean"` (remplissez les échantillons manquants avec la moyenne de la colonne cible).
   * **Remarque :** Il y a plus d'options de « stratégie » et de remplissage dans la documentation [`SimpleImputer()`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html ).

In [46]:
from sklearn.impute import SimpleImputer

# Créer une variable  categorical imputer
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

# Créer une colonnne Door imputer
door_imputer = SimpleImputer(strategy="constant", fill_value=4)

# Créer une colonne Odometer (KM) imputer
num_imputer = SimpleImputer(strategy="mean")

Imputers créés !

Définissons maintenant les colonnes sur lesquelles nous souhaitons imputer.

Pourquoi?

Parce que nous en aurons besoin sous peu (je l'expliquerai dans la cellule de texte suivante).

In [47]:
# Définir différentes fonctionnalités/features de colonnes
categorical_features = ["Make", "Colour"]
door_feature = ["Doors"]
numerical_feature = ["Odometer (KM)"]

Colonnes définies !

Maintenant, comment pourrions-nous transformer nos colonnes ?

Astuce : nous pouvons utiliser la classe [`sklearn.compose.ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) de Scikit-Learn, de la même manière à ce que nous avons fait auparavant pour obtenir nos données à toutes les valeurs numériques.

C'est la raison pour laquelle nous avons défini les colonnes que nous souhaitons transformer.

Nous pouvons donc utiliser la classe `ColumnTransformer()`.

`ColumnTransformer()` prend en entrée une liste de tuples sous la forme `(name_of_transform, transformer_to_use, columns_to_transform)` spécifiant quelles colonnes transformer et comment les transformer.

Par exemple:

```python
imputer = ColumnTransformer([
     ("cat_imputer", cat_imputer, categorical_features)
])
```

Dans ce cas, les variables du tuple sont :
* `name_of_transform` = `"cat_imputer"`
* `transformer_to_use` = `cat_imputer` (l'instance de `SimpleImputer()` que nous avons défini ci-dessus)
* `columns_to_transform` = `categorical_features` (la liste des fonctionnalités catégorielles que nous avons définies ci-dessus).

Développons cela en étendant l'exemple.

In [48]:
from sklearn.compose import ColumnTransformer

# Créez une série de transformations de colonnes à effectuer
imputer = ColumnTransformer([
    ("cat_imputer", cat_imputer, categorical_features),
    ("door_imputer", door_imputer, door_feature),
    ("num_imputer", num_imputer, numerical_feature)])

Bon!

L'étape suivante consiste à adapter notre instance `ColumnTransformer()` (`imputer`) aux données de formation et à transformer les données de test.

En d’autres termes, nous voulons :
1. Apprenez les valeurs d'imputation de l'ensemble de formation.
2. Remplissez les valeurs manquantes dans l'ensemble d'entraînement avec les valeurs apprises en 1.
3. Remplissez les valeurs manquantes dans l'ensemble de test avec les valeurs apprises en 1.

Pourquoi de cette façon ?

Dans notre cas, nous ne calculons pas beaucoup de variables (à l'exception de la moyenne de la colonne « Odomètre (KM) »), cependant, n'oubliez pas que l'ensemble de test doit toujours rester sous forme de données invisibles.

Ainsi, **lorsque vous remplissez des valeurs dans l'ensemble de test, elles ne doivent contenir que des valeurs calculées ou imputées à partir des ensembles d'entraînement**.

Nous pouvons réaliser les étapes 1 et 2 simultanément avec le [`ColumnTransformer.fit_transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer. fit_transform) (`fit` = trouver les valeurs à remplir, `transform` = les remplir).

Et puis nous pouvons effectuer l'étape 3 avec le [`ColumnTransformer.transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer.transform ) (nous voulons uniquement transformer l’ensemble de test, pas apprendre différentes valeurs à remplir).

In [49]:
# Trouver des valeurs pour remplir et transformer les données d'entraînement
filled_X_train = imputer.fit_transform(X_train)

# Remplissez les valeurs de l'ensemble de test avec les valeurs apprises de l'ensemble d'entraînement
filled_X_test = imputer.transform(X_test)

# Vérifiez rempli filled_X_train
filled_X_train

array([['Honda', 'White', 4.0, 71934.0],
       ['Toyota', 'Red', 4.0, 162665.0],
       ['Honda', 'White', 4.0, 42844.0],
       ...,
       ['Toyota', 'White', 4.0, 196225.0],
       ['Honda', 'Blue', 4.0, 133117.0],
       ['Honda', 'missing', 4.0, 150582.0]], dtype=object)

Merveilleux!

Transformons maintenant nos tableaux `filled_X_train` et `filled_X_test` en DataFrames pour inspecter leurs valeurs manquantes.

In [50]:
# Récupérez notre tableau de données transformé dans DataFrame
filled_X_train_df = pd.DataFrame(filled_X_train, 
                                 columns=["Make", "Colour", "Doors", "Odometer (KM)"])

filled_X_test_df = pd.DataFrame(filled_X_test, 
                                columns=["Make", "Colour", "Doors", "Odometer (KM)"])

# Vérifier les données manquantes dans l'ensemble d'entraînement
filled_X_train_df.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

Et y a-t-il des données manquantes dans l'ensemble de test ?

In [51]:
# Vérifiez les données manquantes dans l'ensemble de test
filled_X_test_df.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

Et l'original ?

In [52]:
# Vérifiez l'original... il manque toujours des valeurs
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

Parfait!

Fini les valeurs manquantes !

Mais attendez...

Nos données sont-elles toutes numériques ?

In [53]:
filled_X_train_df.head()

,Make,Colour,Doors,Odometer (KM)
0,Honda,White,4.0,71934.0
1,Toyota,Red,4.0,162665.0
2,Honda,White,4.0,42844.0
3,Honda,White,4.0,195829.0
4,Honda,Blue,4.0,219217.0


Ahh... looks like our `Make` and `Colour` columns are still strings.

Let's one-hot encode them along with the `Doors` column to make sure they're numerical, just as we did previously.

In [54]:
# Maintenant, encodons à chaud les fonctionnalités avec le même code qu'avant
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ["Make", "Colour", "Doors"]

one_hot = OneHotEncoder()

transformer = ColumnTransformer([("one_hot", 
                                  one_hot, 
                                  categorical_features)],
                                remainder="passthrough",
                                sparse_threshold=0) # renvoie ou non une matrice sparse

# Remplissez les valeurs de train et de test séparément
transformed_X_train = transformer.fit_transform(filled_X_train_df)
transformed_X_test = transformer.transform(filled_X_test_df)

transformed_X_train

array([[0.0, 1.0, 0.0, ..., 1.0, 0.0, 71934.0],
       [0.0, 0.0, 0.0, ..., 1.0, 0.0, 162665.0],
       [0.0, 1.0, 0.0, ..., 1.0, 0.0, 42844.0],
       ...,
       [0.0, 0.0, 0.0, ..., 1.0, 0.0, 196225.0],
       [0.0, 1.0, 0.0, ..., 1.0, 0.0, 133117.0],
       [0.0, 1.0, 0.0, ..., 1.0, 0.0, 150582.0]], dtype=object)

Bon!

Maintenant, nos données sont :
1. Tout numérique
2. Aucune valeur manquante

Essayons d'adapter un modèle !

In [55]:
# Maintenant que nous avons transformé X, voyons si nous pouvons adapter un modèle
np.random.seed(42)
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

# Assurez-vous d'utiliser les données transformées (données X remplies et codées One-hot)
model.fit(transformed_X_train, y_train)
model.score(transformed_X_test, y_test)

0.21229043336119102

Vous avez peut-être remarqué que ce résultat est légèrement différent de celui d'avant.

Pourquoi pensez-vous cela est?

C'est parce que nous avons créé nos ensembles de formation et de test différemment.

Nous divisons les données en ensembles d'entraînement et de test *avant* de remplir les valeurs manquantes.

Auparavant, nous faisions l'inverse, remplissions les valeurs manquantes *avant* de diviser les données en ensembles d'entraînement et de test.

Cela peut entraîner une fuite d’informations de l’ensemble de formation dans l’ensemble de test.

N'oubliez pas que l'un des concepts les plus importants de l'apprentissage automatique est de vous assurer que votre modèle ne voit *aucune* données de test avant l'évaluation.

Nous allons continuer à nous entraîner, mais pour l'instant, voici quelques-uns des principaux points à retenir :
* Gardez vos ensembles de formation et de test séparés.
* La plupart des ensembles de données que vous rencontrerez ne seront pas sous une forme prête à commencer immédiatement à les utiliser avec des modèles d'apprentissage automatique. Et certains peuvent nécessiter plus de préparation que d’autres pour être prêts à être utilisés.
* Pour la plupart des modèles d'apprentissage automatique, vos données doivent être numériques. Cela impliquera de convertir tout ce avec quoi vous travaillez en nombres. Ce processus est souvent appelé **ingénierie des fonctionnalités** ou **encodage des fonctionnalités**.
* Certains modèles de machine learning ne sont pas compatibles avec les données manquantes. Le processus de remplissage des données manquantes est appelé **imputation de données**.